# **Proyecto Final BD_NR**

> **¿Qué hace este programa?** Realiza una descarga de la API del Art Institute of Chicago de hasta 1000 obras para descargarla a MongoDB, y luego, usando una máquina virtual, prender una instancia y pasar los datos a Neo4j.

**INSTRUCCIONES** Corre la celda a continuación, llena los inputs deseados, y deja correr el resto de las celdas

## **1. Lectura de inputs**

In [1]:
from IPython.display import display
import ipywidgets as widgets

name_bd_mongo = widgets.Text(value='', description='BD:', layout=widgets.Layout(width='600px'))
name_col_mongo = widgets.Text(value='', description='Colección:', layout=widgets.Layout(width='600px'))
user_neo4j_box = widgets.Text(value='', description='User Neo4j:', layout=widgets.Layout(width='600px'))
password_neo4j_box = widgets.Text(value='', description='Password Neo4j:', layout=widgets.Layout(width='600px'))
local_box = widgets.Text(value='', description='Public IPv4 AWS:', layout=widgets.Layout(width='600px'))
submit_button = widgets.Button(description='Submit')

def submit_button_clicked(b):
    global bd_mongo, col_mongo, user_neo, password_neo, localhost
    bd_mongo = name_bd_mongo.value
    col_mongo = name_col_mongo.value
    user_neo = user_neo4j_box.value
    password_neo = password_neo4j_box.value
    localhost = local_box.value
    print("BD de Mongo:", bd_mongo)
    print("Colección de Mongo:", col_mongo)
    print("Usuario de Neo4j:", user_neo)
    print("Password de Neo4j:", password_neo)
    print("Public IPv4 de instancia:", localhost)

submit_button.on_click(submit_button_clicked)

input_widgets = widgets.VBox([name_bd_mongo, name_col_mongo,  user_neo4j_box, password_neo4j_box, local_box,submit_button])
display(input_widgets)
 

## **2. Import de bibliotecas**

In [8]:
from pprint import pprint
import requests
import pymongo
from neo4j import GraphDatabase

## **3. Conexión API a MongoDB**

Conexión a la API del Art Institute de Chicago para descargar 1000 obras de arte en una base de MongoDB.

**Descarga de obras de la API**

In [ ]:
# Crear un arreglo para guardar las obras de arte

url = "https://api.artic.edu/api/v1/artworks?limit=100" # Descargar 500 obras
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f'Error en la descarga {response.status_code}')
    
# Descargar solo los datos
collection = data['data']

In [ ]:
collection[1]

**Revisar la estructura de cada JSON de una obra**

**Subir las obras a una nueva BD y colección en MongoDB**

In [2]:
# Crear un cliente de MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# Poner nombre a la BD
db = client[bd_mongo]

# Poner nombre a la conexión
my_collection = db[col_mongo]
insert_result = my_collection.insert_many(my_collection)

NameError: name 'bd_mongo' is not defined

## **4. Queries de MongoDB**

In [5]:
# Crear un cliente de MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')

# Poner nombre a la BD
db = client['museum']

# Poner nombre a la conexión
my_collection = db['artwork']

En esta sección, incluimos tres queries avanzados para nuestra BD de Mongo.

**Query 1**

> Explicación: 

In [24]:
# pipeline del query

pipeline1 = [
    {
        '$group': {
            '_id': {
                'artist': '$artist_title'
            },
            'conteo_estilos': {
                '$addToSet': '$style_title'
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'artista': '$_id.artist',
            'conteo_estilos': { '$size': '$conteo_estilos' }
        }
    },
    {
        '$sort': {
            'conteo_estilos': -1,
        }
    },{
        '$limit': 5
    }
]

# Query

result1 = my_collection.aggregate(pipeline1)

Print de los resultados

In [25]:
for document in result1:
    pprint(document)


{'artista': None, 'conteo_estilos': 3}
{'artista': 'Vija Celmins', 'conteo_estilos': 2}
{'artista': 'Hilaire Germain Edgar Degas', 'conteo_estilos': 2}
{'artista': "Georgia O'Keeffe", 'conteo_estilos': 2}
{'artista': 'Édouard Manet', 'conteo_estilos': 2}


**Query 2**

> Explicación: por cada artista, enlistar todos los años de publicación de una obra.

In [14]:
pipeline2 = [
    {"$group": {"_id": "$artist_title", "fechas_display": {"$push": "$date_display"}}},
    {"$project": {"_id": 0, "artista": "$_id", "fechas_display": 1}},
    {"$sort": {"artista": 1}}
]

result2 = my_collection.aggregate(pipeline2)


Print de los resultados

In [15]:
for doc in result1:
    pprint(doc)

{'artista': None,
 'fechas_display': ['17th century',
                    'first half of 17th century',
                    'c. 1570/90',
                    '1570/80',
                    '1550',
                    'late 16th century/early 17th century',
                    '17th century',
                    'c. 10th/11th century',
                    '10th century',
                    '1963, first entry dated 1970',
                    'c. 10th/11th century']}
{'artista': 'Aleksandr Mikhailovich Rodchenko', 'fechas_display': ['1936']}
{'artista': 'Ancient Egyptian', 'fechas_display': ['Late Period (664–332 BCE)']}
{'artista': 'Ancient Roman', 'fechas_display': ['about 1st century CE']}
{'artista': 'Baccarat Glassworks',
 'fechas_display': ['c. 1845–60',
                    'c. 1848–55',
                    'c. 1848–55',
                    'c. 1848–55',
                    'c. 1846–55']}
{'artista': 'Balthus', 'fechas_display': ['1943']}
{'artista': 'Berthe Morisot', 'fechas_displ

**Query 3**

> Explicación: ordenar de menor a mayor, el promedio del ancho de las pinturas de cada artista

In [38]:
pipeline3 = [
    {
        '$group': {
            '_id': {
                'artista': '$artist_title'
            },
            'promedio_tamaños': {
                '$avg': '$thumbnail.width'
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'artista': '$_id.artista',
            'promedio_tamaño': '$promedio_tamaños'
        }
    },
    {
        '$sort': {
            'promedio_tamaño': 1,
        }
    }
]

result3 = my_collection.aggregate(pipeline3)


In [39]:
for i in result3:
    pprint(i)

{'artista': 'Bill Brandt', 'promedio_tamaño': None}
{'artista': 'Sonia Delaunay-Terk', 'promedio_tamaño': 1115.0}
{'artista': 'Moche', 'promedio_tamaño': 1500.0}
{'artista': 'Guido Reni', 'promedio_tamaño': 1581.0}
{'artista': 'Eugène Delacroix', 'promedio_tamaño': 1921.5}
{'artista': 'Georges Braque', 'promedio_tamaño': 2035.5}
{'artista': 'Marc Riboud', 'promedio_tamaño': 2065.0}
{'artista': 'Pierre Bonnard', 'promedio_tamaño': 2221.0}
{'artista': 'Peer Smed', 'promedio_tamaño': 2250.0}
{'artista': 'Compagnie de Saint Louis', 'promedio_tamaño': 2289.5}
{'artista': 'Baccarat Glassworks', 'promedio_tamaño': 2333.4}
{'artista': 'Clichy Glasshouse', 'promedio_tamaño': 2388.0}
{'artista': 'Paul Strand', 'promedio_tamaño': 2436.0}
{'artista': 'Jack Whitten', 'promedio_tamaño': 2617.0}
{'artista': 'John Henry Dearle', 'promedio_tamaño': 2629.0}
{'artista': 'Hilaire Germain Edgar Degas',
 'promedio_tamaño': 2850.3333333333335}
{'artista': 'Dorothy Liebes', 'promedio_tamaño': 2928.0}
{'artist

## **4. Conexión de MongoDB a Neo4j**

Conexión de los datos de MongoDB a Neo4j, para así poder crear una nueva base de grafos, relacionando obras de arte entre ellas.

In [3]:
# Conectarse a Neo4j

link = 'bolt://' + localhost + ':7687'

neo_driver = GraphDatabase.driver('bolt://localhost:7687', auth=(user_neo, password_neo))

### **4.1. Conseguir entidades**

In [6]:
# Artistas
pipeline_artists = [
    {"$group": {"_id": "$artist_title"}},
    {"$project": {"_id": 0, "artista": "$_id"}},
    {"$sort": {"artista": 1}}
]

artistas = my_collection.aggregate(pipeline_artists)

# Obras

pipeline_obras = [{ "$project": {
      "nombre": "$title",
      "id": "$id",
      "lugar_origen": "$place_of_origin",
      "artista": "$artist_title",
      "estilo": "$style_title",
      "tipo": "$medium_display",
      "_id": 0
    }
  }
]


obras = my_collection.aggregate(pipeline_obras)


# Lugares

pipeline_lugares = [
  {"$group": {"_id": "$place_of_origin"}},
  {"$project":{"_id":0, "lugar":"$_id"}},
  {"$sort":{"lugar":1}}
]


lugares = my_collection.aggregate(pipeline_lugares)


# Estilo

pipeline_estilos = [
  {"$group": {"_id": "$style_title"}},
  {"$project":{"_id":0, "estilo":"$_id"}},
  {"$sort":{"estilo":1}}
]


estilos = my_collection.aggregate(pipeline_estilos)
